In [1]:
import quandl
quandl.ApiConfig.api_key = "idNBU8ZQFvkUE9fDgWyz"
import numpy as np
import pandas as pd

In [2]:
prices = quandl.get_table('SHARADAR/SEP', ticker=['AAPL', 'NKE'], date={'gte':'2000-01-01', 'lte':'2018-12-31'}, paginate=True)

In [3]:
pivoted = prices.pivot(index='date', columns='ticker', values='close')

In [4]:
pivoted.head()

ticker,AAPL,NKE
date,,
2018-09-04,228.36,79.60
2018-09-05,226.87,79.92
2018-09-06,223.10,80.40
2018-09-07,221.30,80.30
2018-09-10,218.33,82.10


In [5]:
w = {'AAPL':.75, 'NKE':.25}

In [6]:
cov_matrix = pivoted.pct_change().apply(lambda x: np.log(1+x)).cov()
cov_matrix

ticker,AAPL,NKE
ticker,,
AAPL,0.000720,0.000131
NKE,0.000131,0.000358


In [7]:
df = pivoted.pct_change().apply(lambda x: np.log(1+x))
df.head()

ticker,AAPL,NKE
date,,
2000-01-03,NaN,NaN
2000-01-04,-0.088058,-0.055898
2000-01-05,0.014373,0.056064
2000-01-06,-0.090395,-0.005166
2000-01-07,0.046359,0.000000


# Portfolio Variance

$$\sigma^2(R_p) = w_1w_1\sigma^2(R_1) + w_1w_2COV(R_1,R_2) + w_2w_2\sigma^2(R_2) + w_2w_1COV(R_1,R_2)$$

$$\sigma^2(R_p) = w_1w_1\sigma^2(R_1) + w_2w_2\sigma^2(R_2) + 2w_1w_2COV(R_1,R_2)$$

# Correlation & Covariance

$$p(R_i,R_j) = \frac{COV(R_i, R_j)}{\sigma_i \sigma_j}$$

$$COV(R_i,R_j) = p(R_i,R_j)\sigma_i\sigma_j$$

# Portfolio Variance

$$\sigma^2(R_p) = w_1^2\sigma^2(R_1) + w_2^2\sigma^2(R_2) + 2w_1w_2p(R_i,R_j)\sigma_i \sigma_j$$

In [8]:
sd = df.std()
sd

ticker
AAPL    0.026828
NKE     0.018922
dtype: float64

In [9]:
df['AAPL'].corr(df['NKE'])

0.25894605920611474

In [11]:
p_var = w['AAPL']**2*sd['AAPL']**2+w['NKE']**2*sd['NKE']**2+2*w['AAPL']*w['NKE']*.25*sd['AAPL']*sd['NKE']
p_var

0.0004748118645729359

In [13]:
p_sd = np.sqrt(p_var)
p_sd

0.02179017816753539

In [14]:
ann_p_sd = p_sd*np.sqrt(250)
ann_p_sd

0.34453296815142953

In [17]:
np.linspace(-1,1,10)

array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])

In [18]:
p_vol = []
for corr in np.linspace(-1,1,10):
    p_var = w['AAPL']**2*sd['AAPL']**2+w['NKE']**2*sd['NKE']**2+2*w['AAPL']*w['NKE']*corr*sd['AAPL']*sd['NKE']
    ann_p_sd = np.sqrt(p_var)*np.sqrt(250)
    p_vol.append(ann_p_sd)
p_vol

[0.24333944259471205,
 0.26417775367149054,
 0.28348842436248484,
 0.30156506422351687,
 0.31861777912216926,
 0.33480306209076555,
 0.35024119060721937,
 0.36502697545602314,
 0.37923672557280325,
 0.39293293987115824]